# Visualization Examples

In [43]:
import os
import datetime
import geojson
import json

import psycopg2
from shapely.geometry import Point,Polygon,MultiPolygon
from shapely.wkb import loads
from shapely.wkt import dumps, loads
from ipyleaflet import (
    Map, Marker, TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle,
    CircleMarker, GeoJSON, basemaps
)

In [4]:
# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

In [50]:
# center (lat, lon)
okc = [35.4676, -97.5164]
zoom = 7
mapper = Map(center=okc, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)

In [58]:
# Example of overlap lister in SQL.
dml = """
WITH ol
AS
(
	SELECT 	ok.id, STRING_AGG(st.state_name, ', ') AS state_overlaps
	FROM 	public.ok_township AS ok 
		INNER JOIN public.states_us AS st
			ON ST_Intersects(st.shape, ok.shape)
	WHERE	ok.id < 100
	GROUP BY ok.id
)
SELECT 	ol.state_overlaps, ok.id, ok.twplabel, 
		ST_AsGeoJSON(shape) AS jsonshape
FROM 	public.ok_township AS ok
	INNER JOIN ol
		ON ok.id = ol.id 
;

"""

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c:

    try:
        mapper.clear_layers()
        mapper.add_layer(basemaps.OpenStreetMap.Mapnik)
        
        # Execute DML
        cursor = c.cursor()
        cursor.execute(dml)
        c = cursor.fetchall()
        
        for rec in c:
#             print(rec[1])
            shape = geojson.loads(rec[3])
            layer_shape = GeoJSON(data=shape)
            mapper.add_layer(layer_shape)
            
    except Exception as e:
        raise e

print("Completed.")

Completed.


In [51]:
mapper

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [ ]:
mapper.layers